In [1]:
from pycocotools.coco import COCO
import os

data_dir = '../DATA/coco'
annotation_file = os.path.join(data_dir, 'annotations/instances_train2017.json')

# COCOデータセットを読み込む
coco = COCO(annotation_file)

loading annotations into memory...
Done (t=13.46s)
creating index...
index created!


In [2]:
import cv2
import torch
import numpy as np
import torchvision.transforms as transforms

def reshape_func(image_file, target):#sslに入れるための正方形の画像とそれに対応する計算しなおしたbboxを出力する

    IMAGENET_MEAN = [0.485, 0.456, 0.406] #imagenetの正規化
    IMAGENET_STD = [0.229, 0.224, 0.225]
    IMAGENET_SIZE = 224

    #２枚の画像を比べる(余白なし)

    transform1 = transforms.Resize(224)
    transform2 = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((IMAGENET_MEAN), (IMAGENET_STD))
        ])

    # 画像ファイルのパスを指定する

    img = cv2.imread(image_file)#PILだとshapeがよくわからんからcv2で読み込み
    reshaeped_im = transform1(Image.open(image_file))
    transformed_image = transform2(reshaeped_im).to("cuda")#cv2からだとtorchに入んないからPILで読み込み(いらなそう)

    new_width = reshaeped_im.size[0]
    new_height = reshaeped_im.size[1]


    resize_ratio_x = new_width / img.shape[1]
    resize_ratio_y = new_height / img.shape[0]


    # 画像ファイル名から画像IDを取得する
    image_id = None
    for image_info in coco.dataset['images']:
        if image_info['file_name'] == os.path.basename(image_file):
            image_id = image_info['id']
            break

    if image_id is not None:
        # 画像IDに対応するアノテーション情報を取得する
        annotations_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(annotations_ids)

        # BBOXとラベルを表示する
        for annotation in annotations:
            bbox = annotation['bbox']
            resized_bbox = [
            int((bbox[0] * resize_ratio_x) - ((reshaeped_im.size[0] - 224) / 2)),
            int((bbox[1] * resize_ratio_y) - ((reshaeped_im.size[1] - 224) / 2)),
            int(bbox[2] * resize_ratio_x),
            int(bbox[3] * resize_ratio_y)
            ]
            label = coco.loadCats(annotation['category_id'])[0]['name']
            if label == target:
                
                return transformed_image, resized_bbox

In [3]:
dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
model = dinov2_vitb14.to("cuda")

def extract(target, inputs):#抽出する関数
    feature = None

    def forward_hook(module, inputs, outputs):
        # 順伝搬の出力を features というグローバル変数に記録する
        global features
        # 1. detach でグラフから切り離す。
        # 2. clone() でテンソルを複製する。モデルのレイヤーで ReLU(inplace=True) のように
        #    inplace で行う層があると、値がその後のレイヤーで書き換えられてまい、
        #    指定した層の出力が取得できない可能性があるため、clone() が必要。
        features = outputs.detach().clone()

    # コールバック関数を登録する。
    handle = target.register_forward_hook(forward_hook)

    # 推論する
    model.eval()
    model(inputs)

    # コールバック関数を解除する。
    handle.remove()

    return features

Using cache found in /home/yishido/.cache/torch/hub/facebookresearch_dinov2_main
xFormers not available
xFormers not available


In [4]:
def get_id(bbox):#idをとってくるように変更済み。ちゃんととってきているか確認したい場合はcenter_pointsを返すようにする
    # 画像のサイズとグリッドの設定
    image_size = (224, 224)
    grid_size = (16, 16)###################################################################################

    # BBOXの座標

    # グリッドのセルの幅と高さを計算
    cell_width = image_size[0] // grid_size[0]
    cell_height = image_size[1] // grid_size[1]

    # BBOXの範囲内にあるセルの中心点を取得
    center_points = []
    id = []
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            center_x = i * cell_width + cell_width // 2
            center_y = j * cell_height + cell_height // 2
            if bbox[0] <= center_x <= bbox[0] + bbox[2] and bbox[1] <= center_y <= bbox[1] + bbox[3]:
                center_points.append((center_x, center_y))
                id.append([i,j])
    
    return id

In [7]:
def dot_sim(v1, v2):#内積の関数
    return torch.dot(v1, v2) / (torch.linalg.norm(v1) * torch.linalg.norm(v2))

In [8]:
def dot_count(id_1,emb_1,id_2,emb_2):
    
    A = 0 #最大値がbboxの中に履いている個数

    for i in range(len(id_1)):
        n = id_1[i][0] #行
        m = id_1[i][1] #列

        id_emb = n*14+m 
        emb1 = emb_1[id_emb]
        
        inner_product = []
        for j in range(len(emb_2)):
            inner_product.append(dot_sim(emb1,emb_2[j]).item())
        
        inner_product = torch.tensor(inner_product)
        
        max_index = torch.argmax(inner_product).item()

        k = max_index // 14 #最大値に当たるpatchの行
        l = max_index % 14 #最大値に当たるpatchの列

        max_id = [k, l] #maxのid（行，列）

        if max_id in id_2:
            A += 1
    
    if A == 0:
        wariai = 0

    wariai = A/len(id_1)

    return wariai

In [9]:
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

with open('cat.txt', 'r') as file:
    B_C_path = file.read().splitlines()
        
with open('cat_dog.txt', 'r') as file:
    BnC_path = file.read().splitlines()

with open('dog.txt', 'r') as file:
    C_B_path = file.read().splitlines()

In [10]:
#猫ー猫！＿犬

target1 = 'cat'
target2 = 'dog'

allcount=[]

for i in tqdm(range(len(B_C_path))):
    element1 = B_C_path[i]
    for j in range(len(BnC_path)):
        element2 = BnC_path[j]

        image_path1 = f"../DATA/coco/images/train2017/{element1}"
        image_path2 = f"../DATA/coco/images/train2017/{element2}"

        # try:
        im_1 = reshape_func(image_path1, target1) #chanelが１の場合
        im_2 = reshape_func(image_path2, target1)

        # except Exception as e:
        #     continue

        target_module = model.norm
        emb_1 = extract(target_module, im_1[0].unsqueeze(0))[0][1:,]
        emb_2 = extract(target_module, im_2[0].unsqueeze(0))[0][1:,]

        id_in_bbox_1 = get_id(im_1[1])
        id_in_bbox_2 = get_id(im_2[1])
        # print(dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2))

        # try:
        A = dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2) #

        # except Exception as e: #元になるbboxが小さすぎたせいで、idが取れなかった場合
            # continue

        # print(A)
        allcount.append(A)

print(sum(allcount)/len(allcount))

100%|██████████| 10/10 [02:47<00:00, 16.76s/it]

0.15512704517704515


In [11]:
print(sum(allcount)/len(allcount))

0.15512704517704515


In [12]:
#猫ー猫＿犬!

target1 = 'cat'
target2 = 'dog'

allcount=[]

for i in tqdm(range(len(B_C_path))):
    element1 = B_C_path[i]
    for j in range(len(BnC_path)):
        element2 = BnC_path[j]

        image_path1 = f"../DATA/coco/images/train2017/{element1}"
        image_path2 = f"../DATA/coco/images/train2017/{element2}"

        # try:
        im_1 = reshape_func(image_path1, target1) #chanelが１の場合
        im_2 = reshape_func(image_path2, target2)

        # except Exception as e:
        #     continue

        target_module = model.norm
        emb_1 = extract(target_module, im_1[0].unsqueeze(0))[0][1:,]
        emb_2 = extract(target_module, im_2[0].unsqueeze(0))[0][1:,]

        id_in_bbox_1 = get_id(im_1[1])
        id_in_bbox_2 = get_id(im_2[1])
        # print(dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2))

        # try:
        A = dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2) #

        # except Exception as e: #元になるbboxが小さすぎたせいで、idが取れなかった場合
        #     continue

        # print(A)
        allcount.append(A)

print(sum(allcount)/len(allcount))

100%|██████████| 10/10 [02:47<00:00, 16.74s/it]

0.2754913614163614


In [13]:
print(sum(allcount)/len(allcount))

0.2754913614163614


In [14]:
#犬ー猫！＿犬

target1 = 'cat'
target2 = 'dog'

allcount=[]

for i in tqdm(range(len(C_B_path))):
    element1 = C_B_path[i]
    for j in range(len(BnC_path)):
        element2 = BnC_path[j]

        image_path1 = f"../DATA/coco/images/train2017/{element1}"
        image_path2 = f"../DATA/coco/images/train2017/{element2}"

        # try:
        im_1 = reshape_func(image_path1, target2) #chanelが１の場合
        im_2 = reshape_func(image_path2, target1)

        # except Exception as e:
        #     continue

        target_module = model.norm
        emb_1 = extract(target_module, im_1[0].unsqueeze(0))[0][1:,]
        emb_2 = extract(target_module, im_2[0].unsqueeze(0))[0][1:,]

        id_in_bbox_1 = get_id(im_1[1])
        id_in_bbox_2 = get_id(im_2[1])
        # print(dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2))

        # try:
        A = dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2) #

        # except Exception as e: #元になるbboxが小さすぎたせいで、idが取れなかった場合
        #     continue

        # print(A)
        allcount.append(A)

print(sum(allcount)/len(allcount))

100%|██████████| 10/10 [03:40<00:00, 22.05s/it]

0.14963907967032963


In [15]:
print(sum(allcount)/len(allcount))

0.14963907967032963


In [16]:
#犬ー猫＿犬!

target1 = 'cat'
target2 = 'dog'

allcount=[]

for i in tqdm(range(len(C_B_path))):
    element1 = C_B_path[i]
    for j in range(len(BnC_path)):
        element2 = BnC_path[j]

        image_path1 = f"../DATA/coco/images/train2017/{element1}"
        image_path2 = f"../DATA/coco/images/train2017/{element2}"

        # try:
        im_1 = reshape_func(image_path1, target2) #chanelが１の場合
        im_2 = reshape_func(image_path2, target2)

        # except Exception as e:
        #     continue

        target_module = model.norm
        emb_1 = extract(target_module, im_1[0].unsqueeze(0))[0][1:,]
        emb_2 = extract(target_module, im_2[0].unsqueeze(0))[0][1:,]

        id_in_bbox_1 = get_id(im_1[1])
        id_in_bbox_2 = get_id(im_2[1])
        # print(dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2))

        # try:
        A = dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2) #

        # except Exception as e: #元になるbboxが小さすぎたせいで、idが取れなかった場合
        #     continue

        # print(A)
        allcount.append(A)

print(sum(allcount)/len(allcount))

100%|██████████| 10/10 [03:40<00:00, 22.05s/it]

0.3119382957320456


In [17]:
print(sum(allcount)/len(allcount))

0.3119382957320456


In [18]:
#同じ物体同士

target1 = 'cat'
target2 = 'dog'

allcount=[]

for i in tqdm(range(len(B_C_path))):
    element1 = B_C_path[i]
    for j in range(len(B_C_path)):
        element2 = B_C_path[j]

        image_path1 = f"../DATA/coco/images/train2017/{element1}"
        image_path2 = f"../DATA/coco/images/train2017/{element2}"

        # try:
        im_1 = reshape_func(image_path1, target1) #chanelが１の場合
        im_2 = reshape_func(image_path2, target1)

        # except Exception as e:
        #     continue

        target_module = model.norm
        emb_1 = extract(target_module, im_1[0].unsqueeze(0))[0][1:,]
        emb_2 = extract(target_module, im_2[0].unsqueeze(0))[0][1:,]

        id_in_bbox_1 = get_id(im_1[1])
        id_in_bbox_2 = get_id(im_2[1])
        # print(dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2))

        # try:
        A = dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2) #

        # except Exception as e: #元になるbboxが小さすぎたせいで、idが取れなかった場合
            # continue

        # print(A)
        allcount.append(A)

print(sum(allcount)/len(allcount))

100%|██████████| 10/10 [02:46<00:00, 16.65s/it]

0.3972290903540904


In [19]:
print(sum(allcount)/len(allcount))

0.3972290903540904


In [20]:
#同じ物体同士

target1 = 'cat'
target2 = 'dog'

allcount=[]

for i in tqdm(range(len(C_B_path))):
    element1 = C_B_path[i]
    for j in range(len(C_B_path)):
        element2 = C_B_path[j]

        image_path1 = f"../DATA/coco/images/train2017/{element1}"
        image_path2 = f"../DATA/coco/images/train2017/{element2}"

        # try:
        im_1 = reshape_func(image_path1, target2) #chanelが１の場合
        im_2 = reshape_func(image_path2, target2)

        # except Exception as e:
        #     continue

        target_module = model.norm
        emb_1 = extract(target_module, im_1[0].unsqueeze(0))[0][1:,]
        emb_2 = extract(target_module, im_2[0].unsqueeze(0))[0][1:,]

        id_in_bbox_1 = get_id(im_1[1])
        id_in_bbox_2 = get_id(im_2[1])
        # print(dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2))

        # try:
        A = dot_count(id_in_bbox_1, emb_1, id_in_bbox_2, emb_2) #

        # except Exception as e: #元になるbboxが小さすぎたせいで、idが取れなかった場合
            # continue

        # print(A)
        allcount.append(A)

print(sum(allcount)/len(allcount))

100%|██████████| 10/10 [03:39<00:00, 21.97s/it]

0.550209543928294


In [21]:
print(sum(allcount)/len(allcount))

0.550209543928294


### todolist
1.4パターン試す<br>
2.その間codeがあっているかの確認<br>
3.各モデルで実行する<br>
4.論文かく！